# Convert friction surfaces to access surfaces

This notebook merges each of the new speed rasters to the "main" friction surface for the analysis extent area, generating a new friction surface specific to that road. We use that friction surface to generate accessibility surfaces specific to that road

In [1]:
import os, sys
from datetime import date

import common_rasterio_ops as rast_ops

import numpy as np
import re

import rasterio
from rasterio import features, transform
from rasterio.merge import merge as merge
from rasterio.transform import Affine

import rio_cogeo
from rio_cogeo.cogeo import cog_translate

import pandas as pd
import geopandas as gpd

import shapely
from shapely.geometry import shape, box, MultiPoint, Point, Polygon

import skimage.graph as graph
sys.path.append('../../src/')
import GOSTnets.GOSTNets as gn
import GOSTNets_Raster.src.GOSTNets_Raster.market_access as ma
from gostrocks.src.GOSTRocks.misc import tPrint

## Setup

Today's date

In [2]:
today = date.today().strftime("%d%m%y")

Desired resolution of input and output rasters

In [9]:
res = '31m'

Directories

In [11]:
geo_dir = r'P:\PAK\GEO'
data_dir = r'../../data'

dest_dir = r'destinations'
fric_dir = f'friction//{res}'
acc_dir = r'access'

Projections

In [4]:
# change this to whatever the desired output projection is
dest_crs = 'EPSG:32642'

Load in KP as clipping object

In [5]:
aoi = gpd.read_file(os.path.join(geo_dir,'Boundaries/OCHA/pak_admbnda_adm1_ocha_pco_gaul_20181218.shp'))
aoi = aoi[aoi['ADM1_EN'] == 'Khyber Pakhtunkhwa']
aoi = aoi.to_crs(dest_crs)

# Buffer the polygon by 20km so we take in nearby markets and roads that may be used
aoi.geometry = aoi.buffer(20000)

Destination files

In [6]:
# Use destinations prepared previously. This assumes they are in WGS84 and need to be reprojected to the project's metric CRS

dest_fils = {
    re.findall(r'KP_(.*?).gpkg',fil)[0]: gpd.clip(gpd.read_file(os.path.join(data_dir,dest_dir,fil))\
                                                  .set_crs(4326).to_crs(dest_crs),aoi) \
    for fil in os.listdir(os.path.join(data_dir,dest_dir)) if fil.endswith(".gpkg")
}

In [7]:
len(dest_fils)

24

Slight adjustment to GOST's TT code: export rasters as Float32 to reduce file sizes

In [8]:
# adjust GOST's code to always export float32 rasters, as a space saving measure

def calculate_travel_time_slim(inH, mcp, destinations, out_raster = ''):
    ''' Calculate travel time raster
    
    INPUTS
        inH [rasterio object] - template raster used to identify locations of destinations
        mcp [skimage.graph.MCP_Geometric] - input graph
        destinations [geopandas df] - destinations for nearest calculations
        
    LINKS
        https://scikit-image.org/docs/0.7.0/api/skimage.graph.mcp.html#skimage.graph.mcp.MCP.find_costs
    '''
    # create skimage graph, force costs to float32 to save on space
    cities = ma.get_mcp_dests(inH, destinations)
    costs, traceback = mcp.find_costs(cities)
    costs = costs.astype(np.float32)
    if not out_raster == '':
        meta = inH.meta.copy()
        meta.update(dtype=costs.dtype)
        with rasterio.open(out_raster, 'w', **meta) as out:
            out.write_band(1, costs)
            
    return((costs, traceback))

## Generate access surfaces

Loop over all friction surfaces, for all destinations, to generate access surfaces for each friction surface + each destination using cost distance methods. Note this may take a while!

In [32]:
fric_tifs = sorted([os.path.join(data_dir,fric_dir,file) \
            for file \
            in os.listdir(os.path.join(data_dir,fric_dir)) \
            if file.endswith(".tif")]) # can filter more aggressively here if useful

In [33]:
fric_tifs

['../../data\\friction//31m\\KP_friction_dry_211020_31m_masked_final.tif',
 '../../data\\friction//31m\\KP_friction_msn_211020_31m_masked_final.tif',
 '../../data\\friction//31m\\KP_friction_winter_211020_31m_masked_final.tif']

In [ ]:
# create export dir if missing
if os.path.exists(os.path.join(data_dir,acc_dir,f'current/{today}')) == False:
    os.mkdir(os.path.join(data_dir,acc_dir,f'current/{today}'))
else:
    None

In [34]:
for fric_idx, fric in enumerate(fric_tifs):
    
    fric_name = os.path.basename(fric)
    
    for idx, (key, gdf) in enumerate(dest_fils.items()):

        # countdown
        tPrint(f'{fric_name}: {idx + 1} of {len(dest_fils)}, {key}')

        # define labels
        
        modality = re.split(r'_',fric)[1].replace('_friction_','_')
        season = re.split(r'_',fric)[2]

        # previously had to multiply step2 by * inR.transform.a -- but now not necessary?

        inR = rasterio.open(fric)
        inD = inR.read()[0,:,:] 
        inD = np.nan_to_num(inD,nan=10,posinf=10).astype(np.float32)
        mcp = graph.MCP_Geometric(inD)

        # create travel time raster describing travel time from all cells to destinations
        costs, traceback = calculate_travel_time_slim(inR, mcp, gdf, \
                                                os.path.join(data_dir,acc_dir,f"current/{today}/Current_{modality}_{season}_{key}.tif"))

08:21:33	KP_friction_dry_211020_31m_masked_final.tif: 1 of 24, District_HQs
08:46:41	KP_friction_dry_211020_31m_masked_final.tif: 2 of 24, education_allboys
09:11:24	KP_friction_dry_211020_31m_masked_final.tif: 3 of 24, education_allgirls
09:35:31	KP_friction_dry_211020_31m_masked_final.tif: 4 of 24, education_boys
10:00:16	KP_friction_dry_211020_31m_masked_final.tif: 5 of 24, education_boys_high
10:24:34	KP_friction_dry_211020_31m_masked_final.tif: 6 of 24, education_boys_middle
10:49:02	KP_friction_dry_211020_31m_masked_final.tif: 7 of 24, education_boys_primary
11:13:49	KP_friction_dry_211020_31m_masked_final.tif: 8 of 24, education_girls
11:38:49	KP_friction_dry_211020_31m_masked_final.tif: 9 of 24, education_girls_high
12:04:21	KP_friction_dry_211020_31m_masked_final.tif: 10 of 24, education_girls_middle
12:30:41	KP_friction_dry_211020_31m_masked_final.tif: 11 of 24, education_girls_primary
12:57:02	KP_friction_dry_211020_31m_masked_final.tif: 12 of 24, education_high
13:22:48	KP_

### Convert all access GeoTIFFs to Cloud Optimized GeoTIFFs

Optionally convert all GeoTIFFs to COG GeoTIFFs. COGs are more efficiently tiled and compressed GeoTIFFs optimized to be served out of cloud storage environments. They are notably slimmer than the GeoTIFFs generated by MCP and necessary for our Dask aggregation operations (which leverage XArray)

In [38]:
acc_tifs = sorted([os.path.join(data_dir,acc_dir,f'current/{today}',file) \
            for file \
            in os.listdir(os.path.join(data_dir,acc_dir,f'current/{today}')) \
            if file.endswith(".tif")])

In [39]:
acc_tifs[:3]

['../../data\\access\\current/211021\\Current_dry_District_HQs.tif',
 '../../data\\access\\current/211021\\Current_dry_education_allboys.tif',
 '../../data\\access\\current/211021\\Current_dry_education_allgirls.tif']

In [40]:
for acc_lyr in acc_tifs:
    acc_lyr_cog = str.replace(acc_lyr,'.tif','_COG.tif')
    
    print(acc_lyr)
    !rio cogeo create {acc_lyr} {acc_lyr_cog}

../../data\access\current/211021\Current_dry_District_HQs.tif
../../data\access\current/211021\Current_dry_education_allboys.tif


Reading input: P:/PAK/Code/kpgit_robert/data/access/current/211021/Current_dry_District_HQs.tif
Adding overviews...
Updating dataset tags...
Writing output to: \\sarpov\sarpov\PAK\Code\kpgit_robert\data\access\current\211021\Current_dry_District_HQs_COG.tif


../../data\access\current/211021\Current_dry_education_allgirls.tif


Reading input: P:/PAK/Code/kpgit_robert/data/access/current/211021/Current_dry_education_allboys.tif
Adding overviews...
Updating dataset tags...
Writing output to: \\sarpov\sarpov\PAK\Code\kpgit_robert\data\access\current\211021\Current_dry_education_allboys_COG.tif


../../data\access\current/211021\Current_dry_education_boys.tif


...

Reading input: P:/PAK/Code/kpgit_robert/data/access/current/211021/Current_friction_winter_health_secondary.tif
Adding overviews...
Updating dataset tags...
Writing output to: \\sarpov\sarpov\PAK\Code\kpgit_robert\data\access\current\211021\Current_friction_winter_health_secondary_COG.tif


../../data\access\current/211021\Current_friction_winter_markets_All.tif


Reading input: P:/PAK/Code/kpgit_robert/data/access/current/211021/Current_friction_winter_health_tertiary.tif
Adding overviews...
Updating dataset tags...
Writing output to: \\sarpov\sarpov\PAK\Code\kpgit_robert\data\access\current\211021\Current_friction_winter_health_tertiary_COG.tif


../../data\access\current/211021\Current_friction_winter_markets_Central.tif


Reading input: P:/PAK/Code/kpgit_robert/data/access/current/211021/Current_friction_winter_markets_All.tif
Adding overviews...
Updating dataset tags...
Writing output to: \\sarpov\sarpov\PAK\Code\kpgit_robert\data\access\current\211021\Current_friction_winter_markets_All_COG.tif
Reading input: P:/PAK/Code/kpgit_robert/data/access/current/211021/Current_friction_winter_markets_Central.tif
Adding overviews...
Updating dataset tags...
Writing output to: \\sarpov\sarpov\PAK\Code\kpgit_robert\data\access\current\211021\Current_friction_winter_markets_Central_COG.tif


## Alternatives
Sometimes you only want to generate a surface for one friction surface or group of destinations. The below code does that.

Loop over one friction surface for all destinations

In [ ]:
# specify your season
seas = 'winter'

In [ ]:
#specify your friction surface
prop_fric_pth = f'KP_friction_{seas}_210923_31m_masked.tif'

In [ ]:
for idx, (key, gdf) in enumerate(dest_fils.items()):
    
    # countdown
    tPrint(f'{idx + 1} of {len(dest_fils)}, {key}')
    
    # define desired friction surface
    
    single_fric_fil = os.path.join(data_dir,fric_dir,prop_fric_pth)
    season = re.split(r'_',single_fric_fil)[2]

    # previously had to multiply step2 by * inR.transform.a -- but now not necessary?

    inR = rasterio.open(single_fric_fil)
    inD = inR.read()[0,:,:] 
    inD = np.nan_to_num(inD,nan=10,posinf=10).astype(np.float32)
    mcp = graph.MCP_Geometric(inD)
    
    # create travel time raster describing travel time from all cells to destinations
    costs, traceback = calculate_travel_time_slim(inR, mcp, gdf, \
                                            os.path.join(data_dir,acc_dir,f"current/{today}/Current_{season}_{key}.tif"))

Loop over one destination for all friction surfaces

In [ ]:
# define your single dest

dest = dest_fils[0].values()

# # or a full dict
# single_dest_dict = {key:dest_fils[key] for key in dest_fils.keys() if key == 'health_secondary'}

In [ ]:
prop_fric_pth = f'KP_friction_{seas}_210902_31m_masked_final.tif'

In [ ]:
prop_fric_pth

In [ ]:
# for a single destination

for idx, fric in enumerate(fric_tifs):
    
    # name var for naming output file
    feat_name = os.path.splitext(os.path.basename(fric_tifs[idx]))[0].split('_friction')[0]
    out_acc_name = f'{feat_name}_access.tif'
    
    # countdown
    tPrint(f'{idx + 1} of {len(fric_tifs)}, {feat_name}')
    
    # generate access parameters

    fric = rasterio.open(fric)
    inD = fric.read()[0,:,:]
    inD = np.nan_to_num(inD,nan=10,posinf=10)
    mcp = graph.MCP_Geometric(inD)
    
    # create and export travel time raster describing travel time from all cells to destinations
    
    costs, traceback = calculate_travel_time_slim(fric, mcp, dest, \
                                                os.path.join(data_dir,acc_dir,out_acc_name))